# G-code 분석 API 테스트

FastAPI 서버와 통신하여 G-code 분석 워크플로우를 테스트합니다.

## API 엔드포인트

| 엔드포인트 | 설명 |
|-----------|------|
| `POST /api/v1/gcode/summary` | 요약 분석 시작 (LLM 없음) |
| `GET /api/v1/gcode/analysis/{id}/dashboard` | 대시보드용 플랫 데이터 |
| `GET /api/v1/gcode/analysis/{id}/summary` | 중첩 구조 원본 데이터 |
| `POST /api/v1/gcode/analysis/{id}/error-analysis` | 에러 분석 실행 (LLM) |
| `GET /api/v1/gcode/analysis/{id}/stream` | SSE 실시간 진행상황 |
| `POST /api/v1/gcode/analyze` | 전체 분석 (요약 + 에러) |

## 0. 환경 설정

In [ ]:
import requests
import json
import time
import os

# API 서버 주소
API_BASE_URL = "http://localhost:8000"
GCODE_API_URL = f"{API_BASE_URL}/api/v1/gcode"

print(f"API Base URL: {API_BASE_URL}")
print(f"G-code API URL: {GCODE_API_URL}")

In [ ]:
# 서버 상태 확인
try:
    response = requests.get(f"{GCODE_API_URL}/", timeout=5)
    if response.status_code == 200:
        info = response.json()
        print("[OK] 서버 연결 성공!")
        print(f"  서비스: {info['service']}")
        print(f"  버전: {info['version']}")
        print(f"  설명: {info['description']}")
        print(f"\n사용 가능한 엔드포인트:")
        for name, desc in info['endpoints'].items():
            print(f"  - {name}: {desc}")
    else:
        print(f"[ERROR] 서버 응답 오류: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("[ERROR] 서버에 연결할 수 없습니다.")
    print("\n서버를 먼저 실행하세요:")
    print("  cd c:/Users/USER/factor_AI_python")
    print("  python main.py")

## 1. G-code 파일 로드

In [ ]:
# 테스트할 G-code 파일 경로
GCODE_FILE = r"C:\Users\USER\Downloads\qqqqqqqqqqqqqqqqqq.gcode"

# 파일 존재 확인 및 로드
if os.path.exists(GCODE_FILE):
    file_size = os.path.getsize(GCODE_FILE) / (1024 * 1024)  # MB
    print(f"[OK] 파일 확인됨: {GCODE_FILE}")
    print(f"     크기: {file_size:.2f} MB")
    
    # G-code 내용 읽기
    with open(GCODE_FILE, 'r', encoding='utf-8') as f:
        gcode_content = f.read()
    
    line_count = len(gcode_content.split('\n'))
    print(f"     라인 수: {line_count:,}")
else:
    print(f"[ERROR] 파일을 찾을 수 없습니다: {GCODE_FILE}")
    gcode_content = None

---
# 모드 1: 요약만 분석 (Summary Only)

LLM 호출 없이 빠르게 G-code 요약 정보만 추출합니다.

```
POST /summary → GET /analysis/{id}/dashboard
```

## 1-1. 요약 분석 요청

In [ ]:
# 요약 분석 요청
print("=" * 60)
print("[Step 1] POST /api/v1/gcode/summary")
print("=" * 60)

request_data = {
    "gcode_content": gcode_content,
    "filament_type": None,  # 자동 감지
    "printer_info": None
}

start_time = time.time()
response = requests.post(
    f"{GCODE_API_URL}/summary",
    json=request_data,
    timeout=300
)

if response.status_code == 200:
    result = response.json()
    analysis_id = result['analysis_id']
    print(f"\n[OK] 분석 시작됨")
    print(f"  analysis_id: {analysis_id}")
    print(f"  status: {result['status']}")
    print(f"  mode: {result['analysis_mode']}")
    print(f"  summary_url: {result['summary_url']}")
else:
    print(f"[ERROR] {response.status_code}: {response.text}")
    analysis_id = None

## 1-2. 분석 완료 대기

In [ ]:
# 분석 완료 대기 (폴링)
if analysis_id:
    print("분석 완료 대기 중...")
    
    while True:
        response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}")
        status_data = response.json()
        
        status = status_data['status']
        progress = status_data.get('progress', 0) * 100
        current_step = status_data.get('current_step', '')
        
        print(f"  [{progress:.0f}%] {status} - {current_step}")
        
        if status in ['completed', 'summary_completed', 'error']:
            break
        
        time.sleep(1)
    
    elapsed = time.time() - start_time
    print(f"\n[DONE] 소요 시간: {elapsed:.2f}초")

## 1-3. 대시보드 데이터 조회 (플랫 구조)

In [ ]:
# 대시보드용 플랫 데이터 조회
print("=" * 60)
print("[Step 2] GET /api/v1/gcode/analysis/{id}/dashboard")
print("=" * 60)

if analysis_id:
    response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}/dashboard")
    
    if response.status_code == 200:
        dashboard = response.json()
        
        print(f"\n=== 대시보드 요약 ===")
        print(f"파일: {dashboard['file_name']}")
        print(f"총 라인: {dashboard['total_lines']:,}")
        print(f"슬라이서: {dashboard['slicer_info']}")
        
        print(f"\n--- 시간 ---")
        print(f"예상 출력 시간: {dashboard['print_time_formatted']}")
        
        print(f"\n--- 필라멘트 ---")
        print(f"사용량: {dashboard['filament_used_meters']:.2f}m")
        
        print(f"\n--- 레이어 ---")
        print(f"총 레이어: {dashboard['total_layers']}층")
        print(f"레이어 높이: {dashboard['layer_height_mm']:.2f}mm")
        
        print(f"\n--- 리트랙션 ---")
        print(f"횟수: {dashboard['retraction_count']}회")
        print(f"평균: {dashboard['avg_retraction_mm']:.2f}mm")
        
        print(f"\n--- 서포트 ---")
        print(f"사용: {'예' if dashboard['has_support'] else '아니오'}")
        if dashboard['has_support']:
            print(f"비율: {dashboard['support_ratio_percent']:.1f}%")
        
        print(f"\n--- 속도 (mm/min) ---")
        print(f"범위: {dashboard['speed_min']:.0f} ~ {dashboard['speed_max']:.0f}")
        print(f"평균: {dashboard['speed_avg']:.0f}")
        print(f"이동 속도: {dashboard['travel_speed_avg']:.0f}")
        print(f"출력 속도: {dashboard['print_speed_avg']:.0f}")
        
        print(f"\n--- 온도 (C) ---")
        print(f"노즐: {dashboard['nozzle_temp_min']:.0f} ~ {dashboard['nozzle_temp_max']:.0f} (평균: {dashboard['nozzle_temp_avg']:.1f})")
        print(f"베드: {dashboard['bed_temp_min']:.0f} ~ {dashboard['bed_temp_max']:.0f}")
        print(f"온도 변경: {dashboard['nozzle_temp_changes']}회")
    else:
        print(f"[ERROR] {response.status_code}: {response.text}")

## 1-4. 원본 데이터 조회 (중첩 구조)

In [ ]:
# 중첩 구조 원본 데이터 조회
print("=" * 60)
print("[Step 3] GET /api/v1/gcode/analysis/{id}/summary")
print("=" * 60)

if analysis_id:
    response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}/summary")
    
    if response.status_code == 200:
        summary_data = response.json()
        
        print(f"\nanalysis_id: {summary_data['analysis_id']}")
        print(f"status: {summary_data['status']}")
        print(f"analysis_mode: {summary_data['analysis_mode']}")
        
        # 타임라인 출력
        print(f"\n--- 타임라인 ---")
        for event in summary_data.get('timeline', []):
            status_icon = "[OK]" if event['status'] == 'done' else "[..]" 
            print(f"{status_icon} Step {event['step']}: {event['label']}")
        
        # comprehensive_summary 구조 확인
        cs = summary_data.get('comprehensive_summary', {})
        print(f"\n--- comprehensive_summary 키 ---")
        for key in cs.keys():
            print(f"  - {key}")
    else:
        print(f"[ERROR] {response.status_code}: {response.text}")

---
# 모드 2: 에러 분석 추가

요약 분석 완료 후, 에러 분석을 추가로 실행합니다.

```
POST /analysis/{id}/error-analysis → GET /analysis/{id}
```

In [ ]:
# 에러 분석 실행 (LLM 사용)
print("=" * 60)
print("[Step 4] POST /api/v1/gcode/analysis/{id}/error-analysis")
print("=" * 60)
print("\n[!] LLM API 호출 - 시간이 걸릴 수 있습니다...\n")

if analysis_id:
    start_time = time.time()
    
    response = requests.post(
        f"{GCODE_API_URL}/analysis/{analysis_id}/error-analysis",
        timeout=600
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"[OK] 에러 분석 시작됨")
        print(f"  status: {result['status']}")
        print(f"  message: {result['message']}")
        
        # 완료 대기
        print("\n분석 완료 대기 중...")
        while True:
            status_response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}")
            status_data = status_response.json()
            
            status = status_data['status']
            progress = status_data.get('progress', 0) * 100
            current_step = status_data.get('current_step', '')
            
            print(f"  [{progress:.0f}%] {status} - {current_step}")
            
            if status in ['completed', 'error']:
                break
            
            time.sleep(2)
        
        elapsed = time.time() - start_time
        print(f"\n[DONE] 소요 시간: {elapsed:.2f}초")
    else:
        print(f"[ERROR] {response.status_code}: {response.text}")

In [ ]:
# 에러 분석 결과 조회
if analysis_id:
    response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}")
    
    if response.status_code == 200:
        result = response.json()
        
        print("=== 에러 분석 결과 ===")
        print(f"status: {result['status']}")
        
        # 발견된 이슈
        analysis_result = result.get('result', {})
        issues = analysis_result.get('issues_found', [])
        print(f"\n발견된 이슈: {len(issues)}개")
        
        for i, issue in enumerate(issues[:5]):
            print(f"\n[이슈 {i+1}]")
            print(f"  타입: {issue.get('type', 'N/A')}")
            print(f"  심각도: {issue.get('severity', 'N/A')}")
            desc = issue.get('description', 'N/A')
            print(f"  설명: {desc[:100]}..." if len(desc) > 100 else f"  설명: {desc}")
        
        # 패치 계획
        patch_plan = analysis_result.get('patch_plan')
        if patch_plan:
            print(f"\n--- 패치 계획 ---")
            print(f"총 패치: {patch_plan.get('total_patches', 0)}개")
        
        # 토큰 사용량
        tokens = analysis_result.get('token_usage', {})
        if tokens:
            print(f"\n--- 토큰 사용량 ---")
            print(f"입력: {tokens.get('input_tokens', 0):,}")
            print(f"출력: {tokens.get('output_tokens', 0):,}")
            print(f"총: {tokens.get('total_tokens', 0):,}")

---
# 모드 3: 전체 분석 (한 번에)

요약 + 에러 분석을 한 번에 실행합니다.

```
POST /analyze → GET /analysis/{id}/stream (SSE) → GET /analysis/{id}
```

In [ ]:
# 전체 분석 요청
print("=" * 60)
print("[Full Mode] POST /api/v1/gcode/analyze")
print("=" * 60)
print("\n[!] 전체 분석 - 요약 + 에러 분석 (시간이 걸립니다)...\n")

request_data = {
    "gcode_content": gcode_content,
    "filament_type": None,
    "printer_info": None,
    "analysis_mode": "full"
}

start_time = time.time()
response = requests.post(
    f"{GCODE_API_URL}/analyze",
    json=request_data,
    timeout=300
)

if response.status_code == 200:
    result = response.json()
    full_analysis_id = result['analysis_id']
    print(f"[OK] 전체 분석 시작됨")
    print(f"  analysis_id: {full_analysis_id}")
    print(f"  stream_url: {result['stream_url']}")
else:
    print(f"[ERROR] {response.status_code}: {response.text}")
    full_analysis_id = None

In [ ]:
# 분석 완료 대기 (폴링)
if full_analysis_id:
    print("전체 분석 완료 대기 중...")
    
    prev_step = ""
    while True:
        response = requests.get(f"{GCODE_API_URL}/analysis/{full_analysis_id}")
        status_data = response.json()
        
        status = status_data['status']
        progress = status_data.get('progress', 0) * 100
        current_step = status_data.get('current_step', '')
        
        # 단계가 변경되었을 때만 출력
        if current_step != prev_step:
            print(f"  [{progress:.0f}%] {status} - {current_step}")
            prev_step = current_step
        
        if status in ['completed', 'error']:
            break
        
        time.sleep(2)
    
    elapsed = time.time() - start_time
    print(f"\n[DONE] 총 소요 시간: {elapsed:.2f}초")

In [ ]:
# 전체 분석 결과 조회
if full_analysis_id:
    response = requests.get(f"{GCODE_API_URL}/analysis/{full_analysis_id}")
    
    if response.status_code == 200:
        result = response.json()
        
        print("=== 전체 분석 결과 ===")
        print(f"status: {result['status']}")
        print(f"progress: {result['progress'] * 100:.0f}%")
        
        # 타임라인
        print(f"\n--- 타임라인 ({len(result.get('timeline', []))}개) ---")
        for event in result.get('timeline', [])[-10:]:  # 마지막 10개만
            status_icon = "[OK]" if event['status'] == 'done' else "[..]"
            print(f"{status_icon} {event['label'][:60]}")
        
        # 결과 요약
        analysis_result = result.get('result', {})
        if analysis_result:
            print(f"\n--- 결과 요약 ---")
            print(f"이슈: {len(analysis_result.get('issues_found', []))}개")
            
            patch_plan = analysis_result.get('patch_plan')
            if patch_plan:
                print(f"패치: {patch_plan.get('total_patches', 0)}개")
            
            tokens = analysis_result.get('token_usage', {})
            if tokens:
                print(f"토큰: {tokens.get('total_tokens', 0):,}")

---
# 4. SSE 스트리밍 테스트

실시간 진행 상황을 SSE로 수신합니다.

In [ ]:
# SSE 스트리밍 테스트 (sseclient 필요)
# pip install sseclient-py

try:
    import sseclient
    SSE_AVAILABLE = True
except ImportError:
    print("[!] sseclient가 설치되지 않았습니다.")
    print("    pip install sseclient-py")
    SSE_AVAILABLE = False

In [ ]:
# SSE 스트리밍으로 분석 진행 상황 수신
if SSE_AVAILABLE and gcode_content:
    print("=" * 60)
    print("[SSE] 실시간 스트리밍 테스트")
    print("=" * 60)
    
    # 새 분석 시작
    response = requests.post(
        f"{GCODE_API_URL}/summary",
        json={"gcode_content": gcode_content}
    )
    
    if response.status_code == 200:
        sse_analysis_id = response.json()['analysis_id']
        print(f"analysis_id: {sse_analysis_id}")
        print(f"\nSSE 스트림 연결 중...")
        
        # SSE 연결
        stream_response = requests.get(
            f"{GCODE_API_URL}/analysis/{sse_analysis_id}/stream",
            stream=True
        )
        
        client = sseclient.SSEClient(stream_response)
        
        for event in client.events():
            if event.event == 'timeline':
                data = json.loads(event.data)
                print(f"[Timeline] Step {data['step']}: {data['label']}")
            elif event.event == 'progress':
                data = json.loads(event.data)
                print(f"[Progress] {data['progress']*100:.0f}% - {data['step']}")
            elif event.event == 'complete':
                print("[Complete] 분석 완료!")
                break
            elif event.event == 'error':
                print(f"[Error] {event.data}")
                break

---
# 5. 결과 저장

In [ ]:
# 대시보드 결과 저장
if analysis_id:
    response = requests.get(f"{GCODE_API_URL}/analysis/{analysis_id}/dashboard")
    
    if response.status_code == 200:
        dashboard_data = response.json()
        
        # JSON 저장
        output_path = r"c:\Users\USER\factor_AI_python\tests\api_test_result.json"
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(dashboard_data, f, ensure_ascii=False, indent=2)
        
        print(f"[OK] 결과 저장됨: {output_path}")

---
# 완료!

## API 흐름 요약

### 요약만 (빠름)
```
POST /summary → (대기) → GET /dashboard
```

### 요약 + 에러 분석 (2단계)
```
POST /summary → (대기) → POST /error-analysis → (대기) → GET /analysis/{id}
```

### 전체 분석 (한 번에)
```
POST /analyze → (대기/SSE) → GET /analysis/{id}
```